## **Install and import libraries**

In [1]:
!pip install tokenizers==0.21.0

In [2]:
import math
import os
import re
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer

## **Download and load dataset**

In [ ]:
DATASET_PATH = 'data/poem5_dataset.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [5]:
df['content'][0].split('\n')

['Cái làm ta hạnh phúc',
 'Thực ra cũng chẳng nhiều',
 'Chỉ cần có ai đó',
 'Để ta thầm thương yêu',
 '',
 'Rồi thêm chút công việc',
 'Cho ta làm hàng ngày',
 'Cuối cùng, chút mơ mộng',
 'Để đưa ta lên mây']

## **Build vectorization function**

In [6]:
def text_normalize(text):
    text = text.strip()

    return text

df['content'] = df['content'].apply(lambda x: text_normalize(x))

In [ ]:
def text_generator():
    for text in df['content']:
        yield text

UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"
EOS_TOKEN = "[EOS]"
SOS_TOKEN = "[SOS]"
EOL_TOKEN = "[EOL]"

tokenizer = Tokenizer(WordLevel(unk_token=UNK_TOKEN))
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(special_tokens=[UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, EOL_TOKEN])
tokenizer.train_from_iterator(text_generator(), trainer=trainer)

In [48]:
vocab = tokenizer.get_vocab()
vocab_size = len(vocab)
print("Vocab size:", vocab_size)
print("Vocabulary:")
vocab

Vocab size: 2018
Vocabulary:


{'ập': 2017,
 '!': 14,
 'cười': 223,
 'giãy': 1325,
 'Cuốn': 1048,
 'nga': 757,
 'sâu': 168,
 'rừng': 53,
 'Cát': 1053,
 'ngơ': 428,
 'Của': 1064,
 'xứ': 311,
 'bị': 1247,
 'Chênh': 1034,
 'Thu': 893,
 'Cánh': 872,
 'lọ': 1420,
 'Đường': 546,
 'thuốc': 1588,
 'Mùa': 646,
 'Nâng': 1793,
 'cúi': 324,
 'Đất': 264,
 'mỗi': 1449,
 'lên': 67,
 'phủ': 963,
 'Dường': 1070,
 'tiếng': 139,
 '?”': 1750,
 'cầm': 1841,
 'lành': 499,
 'ríu': 1546,
 'trách': 1966,
 'kỹ': 1388,
 'tư': 1631,
 'Quên': 1163,
 'nến': 1505,
 'lũ': 1407,
 ';': 1012,
 'sát': 1572,
 'ngọn': 152,
 'bờ': 247,
 'thi': 1584,
 'hải': 1362,
 'hình': 1866,
 'vấn': 834,
 'ương': 1735,
 'bảng': 1239,
 'Nào': 464,
 'mù': 506,
 'Hay': 554,
 'giác': 1323,
 'ào': 443,
 'Chứ': 1046,
 '?': 34,
 'giam': 1320,
 'giọng': 1857,
 'Chuyện': 408,
 'hướng': 1361,
 'Một': 26,
 'chuột': 1829,
 'bão': 472,
 'lửng': 1428,
 'Cuộc': 1050,
 'Bộ': 1754,
 'com': 1837,
 'hai': 225,
 'trai': 811,
 'qui': 1528,
 'vai': 984,
 'Chồng': 1043,
 'na': 1450,
 'Ôm': 

In [49]:
default_index = tokenizer.token_to_id("[UNK]")
print("Default index for unknown tokens:", default_index)

Default index for unknown tokens: 0


In [50]:
PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
EOS_TOKEN_ID = tokenizer.token_to_id(EOS_TOKEN)

MAX_SEQ_LEN = 26
tokenizer.enable_padding(length=MAX_SEQ_LEN,
                         pad_id=PAD_TOKEN_ID,
                         pad_token=PAD_TOKEN)
tokenizer.enable_truncation(max_length=MAX_SEQ_LEN)

In [51]:
test_text = df['content'][0].split('\n')[0]
test_encoded = tokenizer.encode(df['content'][0].split('\n')[0])

print("Token IDs:", test_encoded.ids)
print("Tokens:", test_encoded.tokens)
print("Attention Mask:", test_encoded.attention_mask)

Token IDs: [192, 74, 39, 330, 387, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Tokens: ['Cái', 'làm', 'ta', 'hạnh', 'phúc', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention Mask: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [52]:
test_decoded = tokenizer.decode(test_encoded.ids)
print("Decoded text:", test_decoded)

Decoded text: Cái làm ta hạnh phúc


## **Create Poem Dataset**

In [53]:
class PoemDataset(Dataset):
    def __init__(self, df, tokenizer, max_seq_len):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.input_seqs, self.target_seqs, self.attn_masks = self.create_samples(df)

    def split_content(self, content):
        samples = []

        poem_parts = content.split('\n\n')
        for poem_part in poem_parts:
            poem_in_lines = poem_part.split('\n')

            if len(poem_in_lines) == 4:
                samples.append(poem_in_lines)
        return samples

    def prepare_sample(self, sample):
        input_seqs = []
        target_seqs = []
        attn_masks = []

        input_text = "[SOS] " + " [EOL] ".join(sample) + " [EOL] [EOS]"

        unpadded_encoding = self.tokenizer.encode(input_text)
        input_ids = unpadded_encoding.ids

        for idx in range(1, len(input_ids) + 1):
            prefix_ids = input_ids[:idx]
            prefix_text = self.tokenizer.decode(prefix_ids, skip_special_tokens=False)
            prefix_encoding = self.tokenizer.encode(prefix_text)

            target_ids = input_ids[1:idx+1]
            target_text = self.tokenizer.decode(target_ids, skip_special_tokens=False)
            target_encoding = self.tokenizer.encode(target_text)

            input_seqs.append(prefix_encoding.ids)
            target_seqs.append(target_encoding.ids)
            attn_masks.append(prefix_encoding.attention_mask)

        return input_seqs, target_seqs, attn_masks

    def create_samples(self, df):
        all_input_seqs = []
        all_target_seqs = []
        all_attn_masks = []

        for _, row in df.iterrows():
            content = row['content']
            samples = self.split_content(content)
            for sample in samples:
                sample_inputs, sample_targets, sample_attn = self.prepare_sample(sample)
                all_input_seqs.extend(sample_inputs)
                all_target_seqs.extend(sample_targets)
                all_attn_masks.extend(sample_attn)

        all_input_seqs = torch.tensor(all_input_seqs, dtype=torch.long)
        all_target_seqs = torch.tensor(all_target_seqs, dtype=torch.long)
        all_attn_masks = torch.tensor(all_attn_masks, dtype=torch.float)

        return all_input_seqs, all_target_seqs, all_attn_masks

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        return self.input_seqs[idx], self.target_seqs[idx], self.attn_masks[idx]

TRAIN_BS = 512
train_dataset = PoemDataset(
    df=df,
    tokenizer=tokenizer,
    max_seq_len=MAX_SEQ_LEN
)
train_loader = DataLoader(
    train_dataset,
    batch_size=TRAIN_BS,
    shuffle=False
)

In [54]:
input_seqs, target_seqs, attn_masks = next(iter(train_loader))

print(input_seqs[0])
print(target_seqs[0])
print(attn_masks[0])

tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])
tensor([192,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])


In [55]:
for idx in range(MAX_SEQ_LEN):
    print(tokenizer.decode(input_seqs[idx].tolist(), skip_special_tokens=False))
    print(tokenizer.decode(target_seqs[idx].tolist(), skip_special_tokens=False))

[SOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Cái [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[SOS] Cái [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Cái làm [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[SOS] Cái làm [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Cái làm ta [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[SOS] Cái làm ta [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

## **Model**

In [56]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dims, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dims, 2) * (-math.log(10000.0) / embedding_dims))
        pe = torch.zeros(max_len, 1, embedding_dims)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)

        return x

class TransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dims,
        n_heads,
        hidden_dims,
        n_layers,
        dropout=0.5
    ):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.embedding = nn.Embedding(vocab_size, embedding_dims)
        self.embedding_dims = embedding_dims

        self.pos_encoder = PositionalEncoding(embedding_dims, dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            embedding_dims,
            n_heads,
            hidden_dims,
            dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.linear = nn.Linear(embedding_dims, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask=None, attn_mask=None):
        src = self.embedding(src) * math.sqrt(self.embedding_dims)
        src = self.pos_encoder(src)
        if src_mask is None:
            src_mask = nn.Transformer.generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=attn_mask)
        output = self.linear(output)

        return output

In [57]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMS = 128
HIDDEN_DIMS = 64
N_LAYERS = 1
N_HEADS = 16
DROPOUT = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_tests = torch.randint(1, 10, (1, 5)).to(device)

model = TransformerModel(
    VOCAB_SIZE,
    EMBEDDING_DIMS,
    N_HEADS,
    HIDDEN_DIMS,
    N_LAYERS,
    DROPOUT
).to(device)

with torch.no_grad():
    output = model(input_tests)
    print(output.shape)

torch.Size([1, 5, 2018])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


## **Training**

In [58]:
LR = 5.0
EPOCHS = 80

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

In [59]:
model.train()
for epoch in range(EPOCHS):
    losses = []
    for idx, samples in enumerate(train_loader):
        input_seqs, target_seqs, attn_masks = samples
        input_seqs = input_seqs.to(device)
        target_seqs = target_seqs.to(device)
        attn_masks = attn_masks.to(device).permute(1, 0)

        output = model(input_seqs, attn_mask=attn_masks)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target_seqs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f'EPOCH {epoch+1}\tLoss {total_loss}')
    scheduler.step()

EPOCH 1	Loss 4.906108514122341
EPOCH 2	Loss 3.746288838593856
EPOCH 3	Loss 3.4210684506789497
EPOCH 4	Loss 3.2655257971390435
EPOCH 5	Loss 3.0544626816459326
EPOCH 6	Loss 2.7472535112629766
EPOCH 7	Loss 2.3598898545555445
EPOCH 8	Loss 1.9989707936411318
EPOCH 9	Loss 1.6803385019302368
EPOCH 10	Loss 1.435245736785557
EPOCH 11	Loss 1.25024915518968
EPOCH 12	Loss 1.1122795524804487
EPOCH 13	Loss 0.97508414413618
EPOCH 14	Loss 0.881469638451286
EPOCH 15	Loss 0.7902257157408673
EPOCH 16	Loss 0.7130843089974445
EPOCH 17	Loss 0.6500278555828592
EPOCH 18	Loss 0.6024254275404889
EPOCH 19	Loss 0.5693697462911191
EPOCH 20	Loss 0.5365793394005817
EPOCH 21	Loss 0.5106630169827006
EPOCH 22	Loss 0.484585790530495
EPOCH 23	Loss 0.4613220458445342
EPOCH 24	Loss 0.4456090136714604
EPOCH 25	Loss 0.43138690616773523
EPOCH 26	Loss 0.4217233243195907
EPOCH 27	Loss 0.4029278172099072
EPOCH 28	Loss 0.3936356409736302
EPOCH 29	Loss 0.38546241236769635
EPOCH 30	Loss 0.3777641304161238
EPOCH 31	Loss 0.3667556500

## **Inference**

In [20]:
def sample_with_temperature(logits, temperature=1.0):
    if temperature != 1.0:
        logits = logits / temperature

    probabilities = F.softmax(logits, dim=-1)

    sampled_index = torch.multinomial(probabilities, 1).item()

    return sampled_index

In [45]:
model.eval()
temperature = 1.2
input_text = '[SOS] Anh rất là thương em'
tokenizer.no_padding()
tokenizer.no_truncation()
input_encoded = tokenizer.encode(input_text)
input_ids = input_encoded.ids
eos_token_id = tokenizer.token_to_id(EOS_TOKEN)
generated_ids = input_ids.copy()
MAX_GENERATION_LEN = 50
for _ in range(MAX_GENERATION_LEN):
    input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)

    last_token_logits = outputs[0, -1, :]
    next_token_id = sample_with_temperature(last_token_logits, temperature)
    generated_ids.append(next_token_id)

    if next_token_id == eos_token_id:
        break

# Convert the generated tokens back to text
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=False)
generated_text = generated_text.replace(SOS_TOKEN, '')
generated_text = generated_text.replace(EOS_TOKEN, '')
lines = generated_text.split(EOL_TOKEN)
for line in lines:
    print(''.join(line))

 Anh rất là thương em 
 Đẹp - tròn cung bực 
 Mà ở điều bình thường 
 Bao giờ mày say rượu 
 đến trường nhất một chốn 
 Đi rồi qua đời em 
 Cho nhiều nơi để già 
 Đục bản in thơ mày vẫn luôn là mẹ 
 Đi rồi không
